Deliverables : 
    
    1. TextBox++ model
    2. Model can be run through the Utils provided
    3. A .h5 format file can also be used to run (can be done using a pretrained model)
   

In [1]:
import os
import time

In [2]:

os.chdir("/Users/puneethk/Desktop/Projects/Text-Recognition")


In [3]:
import numpy as np
import cv2
import tensorflow.compat.v1 as tf
from timeit import default_timer as timer

# textbox++ models
from modelhelpers.tbpp_model import TBPP512, TBPP512_dense
from modelhelpers.tbpp_utils import PriorUtil

# ssd for help
from modelhelpers.ssd_data import preprocess
from modelhelpers.sl_utils import rbox3_to_polygon, polygon_to_rbox, rbox_to_polygon

Using TensorFlow backend.


In [5]:
if __name__ == '__main__':
    Model = TBPP512_dense
    input_shape = (512,512,3)
    weights_path = './Text-Detection/weights/weights.022.h5'
    confidence_threshold = 0.35
    confidence_threshold = 0.25

    sl_graph = tf.Graph()
    with sl_graph.as_default():
        sl_session = tf.Session()
        with sl_session.as_default():
            sl_model = Model(input_shape)
            prior_util = PriorUtil(sl_model)
            sl_model.load_weights(weights_path, by_name=True)


    input_width = 256
    input_height = 32
    weights_path = './Text-Detection/weights/weights.022.h5'

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
['conv2d_33', 'concatenate_31', 'concatenate_32', 'concatenate_33', 'concatenate_34', 'concatenate_35', 'concatenate_36']


In [6]:
from PIL import Image
start_time = time.time()

# Final TextBox++ Code : (Works on just image)
input_size = input_shape[:2]

# model to predict
img = cv2.imread('./Text-Detection/Pictures/Sample/Test2.jpeg')
IMG = Image.open('./Text-Detection/Pictures/Sample/Test2.jpeg')

print(img.shape)
img_h = img.shape[0]
img_w = img.shape[1]
print(img_h)
print(img_w)

img1 = np.copy(img)
img2 = np.zeros_like(img)

# model to predict
x = np.array([preprocess(img, input_size)])



elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))

#Model start
start_time = time.time()
with sl_graph.as_default():
    with sl_session.as_default():
        y = sl_model.predict(x)

elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))
#Model end


start_time = time.time()
result = prior_util.decode(y[0], confidence_threshold)

if len(result) > 0:
    bboxs = result[:,0:4]
    quads = result[:,4:12]
    rboxes = result[:,12:17]

    boxes = np.asarray([rbox3_to_polygon(r) for r in rboxes])

    xy = boxes
    xy = xy * [img_w, img_h]
    xy = np.round(xy)
    xy = xy.astype(np.int32)
#     print(str(xy))
    
    coords = tuple([xy[0][0][0], xy[0][0][1], xy[0][2][0], xy[0][2][1]])
#     print(str(coords))
    
    cv2.polylines(img1, tuple(xy), True, (0,0,255))
    
    im3 = IMG.convert("RGB")
    im3 = im3.crop(coords)
    im3.save('./Text-Detection/Pictures/Results/Cropped/crop.jpg')
    im3.show()

    rboxes = np.array([polygon_to_rbox(b) for b in np.reshape(boxes, (-1,4,2))])
    bh = rboxes[:,3]
    rboxes[:,2] += bh * 0.1
    rboxes[:,3] += bh * 0.2
    boxes = np.array([rbox_to_polygon(f) for f in rboxes])

    boxes = np.flip(boxes, axis=1) # TODO: fix order of points, why?
    boxes = np.reshape(boxes, (-1, 8))

    boxes_mask_a = np.array([b[2] > b[3] for b in rboxes]) # width > height, in square world
    boxes_mask_b = np.array([not (np.any(b < 0) or np.any(b > 512)) for b in boxes]) # box inside image
    boxes_mask = np.logical_and(boxes_mask_a, boxes_mask_b)

    boxes = boxes[boxes_mask]
    rboxes = rboxes[boxes_mask]
    xy = xy[boxes_mask]

    if len(boxes) == 0:
        boxes = np.empty((0,8))

# draw fps
cv2.rectangle(img1, (0,0), (50, 17), (255,255,255), -1)
cv2.imwrite('./Text-Detection/Pictures/Results/test2.jpg', img1)
print("DONE!")
elapsed_time = time.time() - start_time
print("Performace measure : "+str(elapsed_time))


(360, 810, 3)
360
810
Performace measure : 0.0637807846069336
Performace measure : 2.4237658977508545
DONE!
Performace measure : 0.04191303253173828


variable xy is a 3D array containing the coordinates of box in the order (lefttop, righttop, rightbottom, leftbottom)